<a href="https://colab.research.google.com/github/ayz1070/CapstoneStockModelingProject/blob/main/LSTM_%EB%AA%A8%EB%8D%B8_%EC%A0%81%EC%9A%A9(%EC%88%98%EC%A0%95v5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

In [2]:
kospi = pd.read_excel("kospi100.xlsx", engine='openpyxl')

companies_list = []
companies = kospi['종목명'][:2].tolist()
companies

C:\Users\ahn\anaconda3\envs\CapstonStockProject\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['삼성전자', 'LG에너지솔루션']

In [3]:
# sequence_dataset 만들기
def make_sequence_dataset(feature, label, window_size):

  feature_list = []   # 생성될 feature list
  label_list = []     # 생성될 label list

  for i in range(len(feature)-window_size): # range는 전체값에서 window_size를 뺀 값

    feature_list.append(feature[i:i+window_size]) # feature list 에 i번째서 부터 window size 만큼의 입력데이터를 추가
    label_list.append(label[i+window_size]) # label list 에 그 다음 번째('window_size + 1' 번째)의 정답데이터를 추가

  return np.array(feature_list), np.array(label_list)

In [4]:
#예측값 저장 dict 만들기

day_five = {}
day_ten = {}

In [5]:
for company in companies:
 print(company, '주가 분석중...')
 csv_df = pd.read_csv(f'{company}.csv')  # CSV 파일명 입력
 csv_df = csv_df.loc[:, ['Date', 'Close', 'BPS', 'EPS', '금리', '환율']]
 csv_df = csv_df.set_index('Date') # 날짜를 index로 바꿈
 csv_df.dropna()

 # 날짜 빼고 정규화 작업 진행 -> MinMaxScaler 사용
 scaler = MinMaxScaler()

 # 정규화 수행 -> 정규화된 데이터프레임은 scaled_df
 scale_cols = ['Close', 'BPS', 'EPS', '금리', '환율']
 scaled_df = scaler.fit_transform(csv_df[scale_cols]) # 정규화된 데이터는 넘파이 형태
 scaled_df = pd.DataFrame(scaled_df, columns = scale_cols) # Pandas DataFrame 형태로 변경
 scaled_csv_df = scaled_df.set_index(csv_df.index)

 # 입력데이터 -> 이전날까지의 종가와 변수들
 # 정답데이터 -> 다음날의 종가
 feature_cols = ['Close', 'BPS', 'EPS', '금리', '환율']
 label_cols = ['Close']

 # 입력데이터, 정답데이터 프레임 -> feature_df, label_df
 feature_df = scaled_csv_df[feature_cols]
 label_df = scaled_csv_df[label_cols]

 # DataFrame을 Numpy 형태로 저장
 feature_np = feature_df.to_numpy()
 label_np = label_df.to_numpy()

 window_size = 80  # window_size 만큼의 입력데이터를 이용해 바로 다음 값에 오는 Close 값을 예측
 X, y = make_sequence_dataset(feature_np, label_np, window_size) # X에는 np.array(feature_list), y에는 np.array(label_list) 가 대입됨

 # 모델 훈련을 위한 준비 -> 훈련을 하고, 제대로 훈련이 됐는지 테스트
 split = int(len(X)*0.7) # 테스트 데이터로 분리 -> train:test = 7:3

 # 훈련data 는 전체 데이터의 70퍼센트
 X_train = X[0:split]
 y_train = y[0:split]

 # 테스트data 는 전체 데이터의 30퍼센트
 X_test = X[split:]
 y_test = y[split:]

 ## 5일 예측
 #LSTM 모델 구축
 model = Sequential()

 # 1차원 feature map 생성
 model.add(Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[window_size, 5]))# input_shape = (40,5) -> 다음값 예측을 위한 이전 40개(window_size)의 값과 5개의 특성을 입력으로 넣습니다

 # LSTM layer
 model.add(LSTM(units = 16, activation = 'tanh'))
 model.add(Dense(units = 16, activation = 'sigmoid'))
 model.add(Dense(units = 5)) # 출력층

 # 모델 컴파일
 loss = Huber()
 optimizer = Adam(0.0005)
 model.compile(loss=loss, optimizer=optimizer, metrics=['mse']) # 손실 함수는 Huber, 옵티마이저는 Adam,  평가지표는 mse로 설정

 # 조기종료 설정 -> earlystopping은 10번의 epoch통안 loss 개선이 없다면 학습을 멈춤
 earlystopping = EarlyStopping(monitor='loss', patience=10)


 # 모델 학습 -> epoch은 100번 진행
 model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[earlystopping])

 # 주가 예측 -> test 데이터를 이용하여 학습된 LSTM모델을 테스트
 predictions = model.predict(X_test)

 # 실제값 변환
 # 실제값으로 변화시키기 위해서 기존 데이터에서 종가의 최댓값과 최솟값을 가져옵니다
 close_min = csv_df['Close'].min()
 close_max = csv_df['Close'].max()

 print(csv_df['Close'].min())
 print(csv_df['Close'].max())

 # MinMaxScaler이용해서 실제값으로 역변환
 scaler2 = MinMaxScaler()
 scaled_df2 = scaler2.fit_transform(csv_df[['Close']])

 # MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정
 scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
 scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

 # 예측한 출력값을 실제값으로 역변환
 original_pred_values = scaler2.inverse_transform(predictions)

 # 실제로 에측하기
 # feature_df 에서 가장 최근의 값을 window_size 만큼 가져옴
 pred_feature = feature_df.tail(window_size)

 pred_feature_list = []
 pred_feature_list.append(pred_feature)
 pred_feature = np.array(pred_feature_list)

 # 5일 예측 -> 5일까지의 예측값 5개 출력
 predictions_5d = model.predict(pred_feature)
 pred_values_5d = scaler2.inverse_transform(predictions_5d)


 ##10일 예측
 #LSTM 모델 구축
 model = Sequential()

 # 1차원 feature map 생성
 model.add(Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[window_size, 5]))# input_shape = (40,5) -> 다음값 예측을 위한 이전 40개(window_size)의 값과 5개의 특성을 입력으로 넣습니다

 # LSTM layer
 model.add(LSTM(units = 16, activation = 'tanh'))
 model.add(Dense(units = 16, activation = 'sigmoid'))
 model.add(Dense(units = 10)) # 출력층

  # 모델 컴파일
 loss = Huber()
 optimizer = Adam(0.0005)
 model.compile(loss=loss, optimizer=optimizer, metrics=['mse']) # 손실 함수는 Huber, 옵티마이저는 Adam,  평가지표는 mse로 설정

 # 조기종료 설정 -> earlystopping은 10번의 epoch통안 loss 개선이 없다면 학습을 멈춤
 earlystopping = EarlyStopping(monitor='loss', patience=10)


 # 모델 학습 -> epoch은 100번 진행
 model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[earlystopping])

 # 주가 예측 -> test 데이터를 이용하여 학습된 LSTM모델을 테스트
 predictions = model.predict(X_test)

 # 실제값 변환
 # 실제값으로 변화시키기 위해서 기존 데이터에서 종가의 최댓값과 최솟값을 가져옵니다
 close_min = csv_df['Close'].min()
 close_max = csv_df['Close'].max()

 print(csv_df['Close'].min())
 print(csv_df['Close'].max())

 # MinMaxScaler이용해서 실제값으로 역변환
 scaler2 = MinMaxScaler()
 scaled_df2 = scaler2.fit_transform(csv_df[['Close']])

 # MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정
 scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
 scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

 # 예측한 출력값을 실제값으로 역변환
 original_pred_values = scaler2.inverse_transform(predictions)

 # 실제로 에측하기
 # feature_df 에서 가장 최근의 값을 window_size 만큼 가져옴
 pred_feature = feature_df.tail(window_size)

 pred_feature_list = []
 pred_feature_list.append(pred_feature)
 pred_feature = np.array(pred_feature_list)

 # 10일 예측 -> 10일까지의 예측값 10개 출력
 predictions_10d = model.predict(pred_feature)
 pred_values_10d = scaler2.inverse_transform(predictions_10d)

 pred_values_5d = pred_values_5d.flatten().tolist()
 pred_values_10d = pred_values_10d.flatten().tolist()

 day_five[company] = pred_values_5d
 day_ten[company] = pred_values_10d

 print('5일 예측값', pred_values_5d)
 print('10일 예측값', pred_values_10d)
 print('/n')

삼성전자 주가 분석중...
Epoch 1/100
45/45 [==============================] - 2s 12ms/step - loss: 0.1566 - mse: 0.3149
Epoch 2/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0467 - mse: 0.0935
Epoch 3/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0201 - mse: 0.0403
Epoch 4/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0178 - mse: 0.0355
Epoch 5/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0164 - mse: 0.0329
Epoch 6/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0146 - mse: 0.0293
Epoch 7/100
45/45 [==============================] - 1s 11ms/step - loss: 0.0126 - mse: 0.0253
Epoch 8/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0107 - mse: 0.0214
Epoch 9/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0083 - mse: 0.0166
Epoch 10/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0049 - mse: 0.0099
Epoch 11/100
45/45 [==============

45/45 [==============================] - 1s 12ms/step - loss: 1.7483e-04 - mse: 3.4966e-04
Epoch 82/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7329e-04 - mse: 3.4659e-04
Epoch 83/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7733e-04 - mse: 3.5467e-04
Epoch 84/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7153e-04 - mse: 3.4306e-04
Epoch 85/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7204e-04 - mse: 3.4408e-04
Epoch 86/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7029e-04 - mse: 3.4059e-04
Epoch 87/100
45/45 [==============================] - 1s 12ms/step - loss: 1.6240e-04 - mse: 3.2481e-04
Epoch 88/100
45/45 [==============================] - 1s 13ms/step - loss: 1.6858e-04 - mse: 3.3715e-04
Epoch 89/100
45/45 [==============================] - 1s 12ms/step - loss: 1.6807e-04 - mse: 3.3613e-04
Epoch 90/100
45/45 [==============================] - 1s 13ms/step - loss: 1.

45/45 [==============================] - 1s 12ms/step - loss: 1.8183e-04 - mse: 3.6365e-04
Epoch 60/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7038e-04 - mse: 3.4075e-04
Epoch 61/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7163e-04 - mse: 3.4325e-04
Epoch 62/100
45/45 [==============================] - 1s 12ms/step - loss: 1.8020e-04 - mse: 3.6039e-04
Epoch 63/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7107e-04 - mse: 3.4213e-04
Epoch 64/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7245e-04 - mse: 3.4490e-04
Epoch 65/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7073e-04 - mse: 3.4145e-04
Epoch 66/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7277e-04 - mse: 3.4555e-04
Epoch 67/100
45/45 [==============================] - 1s 12ms/step - loss: 1.6555e-04 - mse: 3.3111e-04
Epoch 68/100
45/45 [==============================] - 1s 12ms/step - loss: 1.

Epoch 38/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0130 - mse: 0.0259
Epoch 39/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0126 - mse: 0.0252
Epoch 40/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0123 - mse: 0.0245
Epoch 41/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0120 - mse: 0.0240
Epoch 42/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0116 - mse: 0.0233
Epoch 43/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0115 - mse: 0.0230
Epoch 44/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0111 - mse: 0.0222
Epoch 45/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0109 - mse: 0.0219
Epoch 46/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0106 - mse: 0.0211
Epoch 47/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0103 - mse: 0.0206
Epoch 48/100
7/7 [==============================] - 0s 12ms/

Epoch 24/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0242 - mse: 0.0484
Epoch 25/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0228 - mse: 0.0456
Epoch 26/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0218 - mse: 0.0435
Epoch 27/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0205 - mse: 0.0410
Epoch 28/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0196 - mse: 0.0392
Epoch 29/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0187 - mse: 0.0375
Epoch 30/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0180 - mse: 0.0359
Epoch 31/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0173 - mse: 0.0347
Epoch 32/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0167 - mse: 0.0334
Epoch 33/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0162 - mse: 0.0323
Epoch 34/100
7/7 [==============================] - 0s 12ms/



---



In [6]:
day_five

{'삼성전자': [61401.58203125,
  62940.91796875,
  63664.2109375,
  60391.5,
  62329.30078125],
 'LG에너지솔루션': [501854.5, 506605.1875, 497651.03125, 505319.25, 503506.875]}

In [14]:
day_ten

{'삼성전자': [64355.46875,
  64390.4453125,
  66777.421875,
  67258.6328125,
  63402.69140625,
  66014.21875,
  65255.99609375,
  63503.16796875,
  66854.7578125,
  67002.9375],
 'LG에너지솔루션': [480550.15625,
  470106.96875,
  477110.34375,
  469598.21875,
  457134.25,
  469516.8125,
  453479.34375,
  481627.71875,
  463291.46875,
  474858.84375]}

In [35]:
import requests

data = {
    'day_five': day_five
}

server_url = "https://eb86-39-118-146-59.ngrok-free.app/day_five"

response = requests.post(server_url, json=data)  # POST 요청으로 변경, 헤더는 자동으로 설정됨

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)

실패: 500


In [34]:
import requests

data = {
    'day_ten': day_ten
}

server_url = 'https://eb86-39-118-146-59.ngrok-free.app/asentiment'

response = requests.post(server_url, json=data)  # POST 요청으로 변경, 헤더는 자동으로 설정됨

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)

성공
